In [ ]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent / "src"))

from src.detection import create_detector
from src.utils import draw_detections, draw_info_panel, FPSMeter
from src.config import PRIMARY_LIVING_BEINGS

print("Libraries imported successfully!")
print(f"Detecting: {list(PRIMARY_LIVING_BEINGS.values())}")


In [ ]:
# Initialize the YOLOv8 detector
detector = create_detector()

print("YOLOv8 detector initialized!")
print("Model loaded from:", detector.model_path)


In [ ]:
# Create a test image (a blank image with some shapes)
test_image = np.zeros((480, 640, 3), dtype=np.uint8)

cv2.rectangle(test_image, (100, 100), (300, 400), (100, 100, 100), -1)  # Gray rectangle
cv2.circle(test_image, (450, 200), 80, (150, 150, 150), -1) # Gray circle
cv2.putText(test_image, "Test Image", (250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

detections = detector.detect_living_beings(test_image)

print(f"🔍 Found {len(detections)} detections")
for i, det in enumerate(detections):
    print(f"  {i+1}. {det['class_name']} (confidence: {det['confidence']:.2f})")
# Display the test image with detections
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
plt.title("Test Image")
plt.axis('off')
plt.show()


In [ ]:
def process_video_demo(video_path, max_frames=10):
    """Process a video and show detection results."""
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return
    
    cap = cv2.VideoCapture(video_path)
    fps_meter = FPSMeter()
    frame_count = 0
    
    print(f"Processing video: {video_path}")
    print("Showing first few frames with detections...")
    
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        # Detect living beings
        detections = detector.detect_living_beings(frame)
        
        result_frame = draw_detections(frame.copy(), detections)
        result_frame = draw_info_panel(
            result_frame, 
            fps=fps_meter.get_fps(),
            detection_count=len(detections)
        )
        
        result_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(result_rgb)
        plt.title(f"Frame {frame_count + 1} - Detections: {len(detections)}")
        plt.axis('off')
        plt.show()
        
        print(f"Frame {frame_count + 1}: {len(detections)} detections")
        for det in detections:
            print(f"  - {det['class_name']}: {det['confidence']:.2f}")
        
        fps_meter.update()
        frame_count += 1
    
    cap.release()
    print(f"Processed {frame_count} frames")


In [ ]:
print("Morph1x Detection Capabilities")
print("=" * 40)
print("Living beings that can be detected:")
for class_id, class_name in PRIMARY_LIVING_BEINGS.items():
    print(f"  {class_id:2d}. {class_name}")

print(f"\nTotal classes: {len(PRIMARY_LIVING_BEINGS)}")
print(f"Primary focus: Humans and animals")
# Performance test on a random image
import time
test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)

start_time = time.time()
detections = detector.detect_living_beings(test_image)
end_time = time.time()

print(f"\nPerformance Test:")
print(f"  Processing time: {(end_time - start_time)*1000:.2f} ms")
print(f"  Detections found: {len(detections)}")

print(f"\nMorph1x is ready for real-time detection!")
